In [1]:
%pip install langchain
%pip install --upgrade --quiet  langchain-openai
%pip install langchainhub
%pip install faiss-cpu
%pip install python-dotenv

  Obtaining dependency information for langchain-core<0.2.0,>=0.1.31 from https://files.pythonhosted.org/packages/43/8b/48b7e6de9041d2b33d5108e154b82d1bd6c47cc68f0e44cb4fcdaccf5ec7/langchain_core-0.1.52-py3-none-any.whl.metadata
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.4
    Uninstalling langchain-core-0.2.4:
      Successfully uninstalled langchain-core-0.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.8 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.52 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use up

In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = DirectoryLoader('data/3-product-info/', glob="*.md", loader_cls=TextLoader, loader_kwargs={'autodetect_encoding': True})
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", openai_api_version="2023-05-15")
vector = FAISS.from_documents(documents, embeddings)


In [4]:
from langchain.tools.retriever import create_retriever_tool

retriever = vector.as_retriever()

In [5]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### Phi-3 mini

In [6]:
from langchain_community.llms import Ollama

model_name = "phi3:mini"
llm = Ollama(model=model_name)

In [7]:
from langchain import hub

# Get the prompt to use - you can modify this!

prompt = hub.pull("rlm/rag-prompt")
prompt.pretty_print()

================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [10]:
question = "Tell me about some fetaures of outdoor living tents."
result = qa_chain({"query": question })

In [11]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

(" OutdoorLiving's TrailMaster X4 Tent is made from durable polyester material "
 'with a spacious interior to accommodate up to eight people. It features '
 'water-resistant construction, mesh panels for ventilation and insect '
 'protection, multiple doors for convenience, reflective guy lines for night '
 'visibility, and an easy freestanding setup. Additionally, it includes a '
 'rainfly for added weather protection and comes with a carrying bag for '
 'convenient storage and transportation.\n'
 '\n'
 "OutdoorLiving's SkyView 2-Person Tent is made from durable materials "
 'suitable for outdoor conditions. It has two large doors, vestibules for '
 'extra gear storage, mesh panels to reduce condensation, reflective guy '
 'lines, and a freestanding design. The tent offers easy setup with '
 'color-coded poles, multiple interior pockets for organization, and comes '
 'with a carrying bag for portability.\n'
 '\n'
 'Both tents are designed for durability, ease of use, weather resistan

### GPT

In [12]:
from langchain_openai import AzureChatOpenAI

# Choose the LLM that will drive the agent
gpt = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4o"
)

In [13]:
gpt_chain = RetrievalQA.from_chain_type(
        gpt,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [14]:
gpt_result = qa_chain({"query": question })

In [15]:
pp.pprint(gpt_result["result"])

(" OutdoorLiving's TrailMaster X4 Tent features a durable polyester material "
 'with spacious interior accommodating up to 8 people, water-resistant '
 'construction, mesh panels for ventilation and insect protection, reflective '
 'guy lines for visibility at night, freestanding design for easy setup, and '
 'an included carry bag for convenient storage. Its technical specs include a '
 'peak height of 6.5 feet, two doors for entry/exit, waterproof rating up to '
 '3000mm, and dimensions of 12ft x 10ft x 7ft when packed.')
